<a href="https://colab.research.google.com/github/SabeehAnsari/Pytorch-Image-classifier/blob/main/Pytorch_Image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:06<00:00, 28.2MB/s]


In [ ]:
image, label = train_data[0]

In [ ]:
image.size()

torch.Size([3, 32, 32])

In [ ]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class NeuralNet(nn.Module):

  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3,12,5)  # (12,28,28)
    self.pool = nn.MaxPool2d(2,2)  # (12,14,14)
    self.conv2 = nn.Conv2d(12,24,5)
    self.fc1 = nn.Linear(24*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(30):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')


Training epoch 0...
Loss: 2.1188
Training epoch 1...
Loss: 1.7020
Training epoch 2...
Loss: 1.5052
Training epoch 3...
Loss: 1.4007
Training epoch 4...
Loss: 1.3072
Training epoch 5...
Loss: 1.2284
Training epoch 6...
Loss: 1.1586
Training epoch 7...
Loss: 1.0912
Training epoch 8...
Loss: 1.0333
Training epoch 9...
Loss: 0.9849
Training epoch 10...
Loss: 0.9379
Training epoch 11...
Loss: 0.9011
Training epoch 12...
Loss: 0.8621
Training epoch 13...
Loss: 0.8294
Training epoch 14...
Loss: 0.7946
Training epoch 15...
Loss: 0.7632
Training epoch 16...
Loss: 0.7361
Training epoch 17...
Loss: 0.7090
Training epoch 18...
Loss: 0.6818
Training epoch 19...
Loss: 0.6590
Training epoch 20...
Loss: 0.6322
Training epoch 21...
Loss: 0.6122
Training epoch 22...
Loss: 0.5845
Training epoch 23...
Loss: 0.5689
Training epoch 24...
Loss: 0.5449
Training epoch 25...
Loss: 0.5217
Training epoch 26...
Loss: 0.5031
Training epoch 27...
Loss: 0.4778
Training epoch 28...
Loss: 0.4659
Training epoch 29...
Los

In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [ ]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 69.7%
